# Day 13

In years past, the holiday feast with your family hasn't gone so well. Not everyone gets along! This year, you resolve, will be different. You're going to find the optimal seating arrangement and avoid all those awkward conversations.

You start by writing up a list of everyone invited and the amount their happiness would increase or decrease if they were to find themselves sitting next to each other person. You have a circular table that will be just big enough to fit everyone comfortably, and so each person will have exactly two neighbors.

For example, suppose you have only four attendees planned, and you calculate their potential happiness as follows:

```text
Alice would gain 54 happiness units by sitting next to Bob.
Alice would lose 79 happiness units by sitting next to Carol.
Alice would lose 2 happiness units by sitting next to David.
Bob would gain 83 happiness units by sitting next to Alice.
Bob would lose 7 happiness units by sitting next to Carol.
Bob would lose 63 happiness units by sitting next to David.
Carol would lose 62 happiness units by sitting next to Alice.
Carol would gain 60 happiness units by sitting next to Bob.
Carol would gain 55 happiness units by sitting next to David.
David would gain 46 happiness units by sitting next to Alice.
David would lose 7 happiness units by sitting next to Bob.
David would gain 41 happiness units by sitting next to Carol.
```

Then, if you seat Alice next to David, Alice would lose 2 happiness units (because David talks so much), but David would gain 46 happiness units (because Alice is such a good listener), for a total change of 44.

If you continue around the table, you could then seat Bob next to Alice (Bob gains 83, Alice gains 54). Finally, seat Carol, who sits next to Bob (Carol gains 60, Bob loses 7) and David (Carol gains 55, David gains 41). The arrangement looks like this:

```text
     +41 +46
+55   David    -2
Carol       Alice
+60    Bob    +54
     -7  +83
```

After trying every other seating arrangement in this hypothetical scenario, you find that this one is the most optimal, with a total change in happiness of 330.

What is the total change in happiness for the optimal seating arrangement of the actual guest list?

## Puzzle 1

In [1]:
from collections import defaultdict
from itertools import permutations
from typing import Dict, List, Tuple


def parse_relation(instr:str) -> Tuple:
    """Return account of gain/loss for a couple
    
    :param instr: input line describing couple's relationship
    """
    data = instr.strip().split()
    
    person1, person2 = data[0], data[-1][:-1]
    val = int(data[3])
    if data[2] == "lose":
        val = val * -1
        
    return (person1, person2), val


def collate_costs(defns:List[str]) -> Dict:
    """Return dictionary of costs for sitting next to each other
    
    :param defns: list of relationship definitions
    """
    costs = defaultdict(int)
    
    for defn in defns:
        rel, val = parse_relation(defn)
        costs[tuple(sorted(rel))] += val
        
    return costs


def calc_cost(names:Tuple, defns:Dict) -> int:
    """Return cost of a specific seating arrangement
    
    :param names: ordered set of names
    :param defns: cost for each pair of names sitting next to each other
    """
    cost = 0
    for _ in range(len(names) - 1):
        key = tuple(sorted([names[_], names[_ + 1]]))
        cost += defns[key]
        
    cost += defns[tuple(sorted([names[-1], names[0]]))]
    
    return cost


def find_best_seating(defns:List[str]):
    """Return best seating arrangement
    
    :param defns: list of relationship definitions
    """
    names = set()
    defns = collate_costs(defns)
    
    for rel in defns.keys():
        names = names.union(set(rel))
    
    options = [(calc_cost(_, defns), _) for _ in permutations(names)]
    
    return sorted(options, reverse=True)[0]

In [2]:
defns = ("Alice would gain 54 happiness units by sitting next to Bob.",
        "Alice would lose 79 happiness units by sitting next to Carol.",
        "Alice would lose 2 happiness units by sitting next to David.",
        "Bob would gain 83 happiness units by sitting next to Alice.",
        "Bob would lose 7 happiness units by sitting next to Carol.",
        "Bob would lose 63 happiness units by sitting next to David.",
        "Carol would lose 62 happiness units by sitting next to Alice.",
        "Carol would gain 60 happiness units by sitting next to Bob.",
        "Carol would gain 55 happiness units by sitting next to David.",
        "David would gain 46 happiness units by sitting next to Alice.",
        "David would lose 7 happiness units by sitting next to Bob.",
        "David would gain 41 happiness units by sitting next to Carol.")

for defn in defns:
    print(defn, parse_relation(defn))
    
costs = collate_costs(defns)
print(costs)

print(find_best_seating(defns))

Alice would gain 54 happiness units by sitting next to Bob. (('Alice', 'Bob'), 54)
Alice would lose 79 happiness units by sitting next to Carol. (('Alice', 'Carol'), -79)
Alice would lose 2 happiness units by sitting next to David. (('Alice', 'David'), -2)
Bob would gain 83 happiness units by sitting next to Alice. (('Bob', 'Alice'), 83)
Bob would lose 7 happiness units by sitting next to Carol. (('Bob', 'Carol'), -7)
Bob would lose 63 happiness units by sitting next to David. (('Bob', 'David'), -63)
Carol would lose 62 happiness units by sitting next to Alice. (('Carol', 'Alice'), -62)
Carol would gain 60 happiness units by sitting next to Bob. (('Carol', 'Bob'), 60)
Carol would gain 55 happiness units by sitting next to David. (('Carol', 'David'), 55)
David would gain 46 happiness units by sitting next to Alice. (('David', 'Alice'), 46)
David would lose 7 happiness units by sitting next to Bob. (('David', 'Bob'), -7)
David would gain 41 happiness units by sitting next to Carol. (('Da

### Solution

In [3]:
with open("day13.txt", "r") as ifh:
    print(find_best_seating(ifh.readlines()))

(618, ('Mallory', 'George', 'David', 'Carol', 'Bob', 'Eric', 'Alice', 'Frank'))


## Puzzle 2

In all the commotion, you realize that you forgot to seat yourself. At this point, you're pretty apathetic toward the whole thing, and your happiness wouldn't really go up or down regardless of who you sit next to. You assume everyone else would be just as ambivalent about sitting next to you, too.

So, add yourself to the list, and give all happiness relationships that involve you a score of 0.

What is the total change in happiness for the optimal seating arrangement that actually includes yourself?

In [4]:
def find_best_self_seating(defns:List[str]):
    """Return best seating arrangement including yourself
    
    :param defns: list of relationship definitions
    """
    names = set()
    defns = collate_costs(defns)
    
    for rel in defns.keys():
        names = names.union(set(rel))
    
    for name in names:
        key = tuple(sorted(["me", name]))
        defns[key] = 0
    
    names.add("me")
    
    options = [(calc_cost(_, defns), _) for _ in permutations(names)]
    
    return sorted(options, reverse=True)[0]

In [5]:
print(find_best_self_seating(defns))

(286, ('me', 'David', 'Carol', 'Bob', 'Alice'))


### Solution

In [6]:
with open("day13.txt", "r") as ifh:
    print(find_best_self_seating(ifh.readlines()))

(601, ('me', 'Eric', 'Alice', 'Frank', 'Mallory', 'George', 'David', 'Carol', 'Bob'))
